<a href="https://colab.research.google.com/github/riccardomarin/SpectralShapeAnalysis/blob/master/Matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

--2020-07-31 07:13:25--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   152MB/s    in 0.4s    

2020-07-31 07:13:26 (152 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
installing: tk-8.6

In [ ]:
import igl
import scipy
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
import meshplot
from scipy.sparse.linalg import eigs,eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

!pip install tensorflow
import tensorflow as tf
tf.__version__

     |████████████████████████████████| 320.4 MB 38 kB/s 
     |████████████████████████████████| 6.8 MB 54.4 MB/s 
     |████████████████████████████████| 57 kB 4.6 MB/s 
     |████████████████████████████████| 1.3 MB 57.0 MB/s 
     |████████████████████████████████| 3.0 MB 50.9 MB/s 
     |████████████████████████████████| 104 kB 53.3 MB/s 
     |████████████████████████████████| 65 kB 3.4 MB/s 
     |████████████████████████████████| 459 kB 51.3 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 26.1 MB 1.3 MB/s 
     |████████████████████████████████| 2.9 MB 36.9 MB/s 
     |████████████████████████████████| 20.1 MB 1.2 MB/s 
     |████████████████████████████████| 88 kB 6.5 MB/s 
     |████████████████████████████████| 91 kB 8.9 MB/s 
     |████████████████████████████████| 61 kB 348 kB/s 
     |████████████████████████████████| 779 kB 39.2 MB/s 
     |████████████████████████████████| 298 kB 44.5 MB/s 
     |████████████████████

'2.2.0'

In [ ]:
def plot_pair(v1, v2, f1, f2, c1, c2, color_ops = {}):
    # Compute a scale factor
    M1 = igl.massmatrix(v1, f1, igl.MASSMATRIX_TYPE_VORONOI)
    M2 = igl.massmatrix(v2, f2, igl.MASSMATRIX_TYPE_VORONOI)
    scale_factor = np.sqrt(np.sum(M2)/np.sum(M1))

    # Align the shapes
    v2 = v2 - np.mean(v2,axis=0)
    v1_align = v1 * scale_factor + np.mean(v1,axis=0) + [0.7,-0.7,0.0]

    # Merge the models
    v_all = np.vstack((v1_align, v2))
    f_all = np.vstack((f1, f2 + np.max(f1)+1))
    c_all = np.vstack((c1, c2))
    
    plot(v_all, f_all, c_all, shading = color_ops)

In [ ]:
# Load Shapes
v_src, f_src = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_089.off"))

L_src = -igl.cotmatrix(v_src, f_src)
M_src = igl.massmatrix(v_src, f_src, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_src, evecs_src = eigsh(L_src, 200, M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_src, evecs_src = eigsh(L_src- 1e-8* scipy.sparse.identity(v_src.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)

evals_src = evals_src.astype(np.float32)
evals_src = evals_src.astype(np.float32)

v_tar, f_tar = igl.read_triangle_mesh(os.path.join('.', "data", "tr_reg_090.off"))

L_tar = -igl.cotmatrix(v_tar, f_tar)
M_tar = igl.massmatrix(v_tar, f_tar, igl.MASSMATRIX_TYPE_VORONOI)

try:
    evals_tar, evecs_tar = eigsh(L_tar, 200, M_tar, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
except:
    evals_tar, evecs_tar = eigsh(L_tar- 1e-8* scipy.sparse.identity(v_tar.shape[0]), 200,
                         M_src, sigma=0.0, which='LM', maxiter=1e9, tol=1.e-15)
    
evals_src = evals_tar.astype(np.float32)
evals_src = evals_tar.astype(np.float32)

ValueError: ignored

# WKS

We will use the Wave Kernel Signature (WKS) descriptor to do the matching. Recall the formula:

$K_E(x,x) = \sum\limits_{l=1}^{\infty}e^{- \frac{(log(E) - log(\lambda_l))^2}{2\sigma^2}} \phi_l(x)^2 $

Where:
- $sigma = 7 \delta$
- $delta =  (e_{max} - e{min})/ M$
- $e_{max} = log(E_N) - 2\sigma$
- $e_{min} = log(E_1) + 2\sigma$
- $E_N$ is the max eigenvalue in absolute value
- $E_1$ is the min non-zero eigenvalue in absolute value
- $M$ is the number of WKS scales

The tasks are:
- Read the meshes, compute the LBO eigenvectors
- Define the WKS computation
- Visualize the WKS scales on meshes
- Perform the matching using WKS (Nearest-Neighbor in the descriptor space)
- Visualize the matching (and compute the error)

Are the descriptors coherent among the shapes, for different descriptor scales? Is the matching good? We can change the number of descriptors: does it impact the matching?

In [ ]:
def WKS(vertices, faces, evals, evecs, wks_size, variance):
    # Number of vertices
    n = vertices.shape[0]
    WKS = np.zeros((n,wks_size))

    # Just for numerical stability
    evals[evals<1e-6] = 1e-6

    # log(E)
    log_E = np.log(evals).T

    # Define the energies step
    e = np.linspace(log_E[1], np.max(log_E)/1.02, wks_size)

    # Compute the sigma
    sigma = (e[1]-e[0]) * variance
    C = np.zeros((wks_size,1))

    for i in np.arange(0,wks_size):
        # Computing WKS
        WKS[:,i] = np.sum(
            (evecs)**2 * np.tile( np.exp((-(e[i] - log_E)**2) / (2*sigma**2)),(n,1)), axis=1)
        
        # Noramlization
        C[i] = np.sum(np.exp((-(e[i]-log_E)**2)/(2*sigma**2)))
        
    WKS = np.divide(WKS,np.tile(C,(1,n)).T)
    return WKS

In [ ]:
# Computing the descriptors for the two shapes
d_src = WKS(v_src, f_src, evals_src, evecs_src, 100, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, 100, 7)

In [ ]:
# Visualizing descriptors
i = 2
plot_pair(v_src, v_tar, f_src, f_tar, d_src[:,i:i+1], d_tar[:,i:i+1])

In [ ]:
# Nearest Neighbor
treesearch = sp.spatial.cKDTree(d_tar)
p2p = treesearch.query(d_src, k=1)[1]

In [ ]:
# To see the quality of the matching we plot a function on one shape and we transfer it to the other
funz_ = (v_tar - np.min(v_tar,0))/np.tile((np.max(v_tar,0)-np.min(v_tar,0)),(np.size(v_tar,0),1));
colors = np.cos(funz_);
funz_tar = (colors-np.min(colors))/(np.max(colors) - np.min(colors));
funz_src = funz_tar[p2p]


In [ ]:
# Plot and (euclidean) error evaluation
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

# Functional Maps

Here we will use the Functional Maps framework. Given some descriptors $d$ on the first shape and $f$ on the second, we will compute our map $C$ such that: \
$ d = \phi_{src}^{T} A_{src} d $ \
$ f = \phi_{tar}^{T} A_{tar} f $ \
$ C D = F $ \
$ C =  F D^{-1} $

In [ ]:
# WKS
n_evals = 5
n_desc = 3
D = (np.matmul(evecs_src[:,0:n_evals].T, np.matmul(M_src.todense() , d_src[:,0:n_desc])))
F = (np.matmul(evecs_tar[:,0:n_evals].T, np.matmul(M_tar.todense() , d_tar[:,0:n_desc])))
C =  np.matmul(F,np.linalg.pinv(D))

plt.imshow(C)
plt.colorbar()
plt.show()

In [ ]:
# Compute p2p correspondence
treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
p2p = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]

In [ ]:
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

In [ ]:
# landmarks
n_land = 45
n_evals = 20
step = np.int(np.ceil(v_src.shape[0] / n_land))
a = np.arange(0,v_src.shape[0],step)

landmarks = np.zeros((v_src.shape[0], a.size))
landmarks[a,np.arange(a.size)] = 1

D = (np.matmul(evecs_src[:,0:n_evals].T, np.matmul(M_src.todense() , landmarks[:,0:n_land]))).T
F = (np.matmul(evecs_tar[:,0:n_evals].T, np.matmul(M_tar.todense() , landmarks[:,0:n_land]))).T
C =  np.matmul(np.linalg.pinv(D),F)

plt.imshow(C)
plt.colorbar()
plt.show()

In [ ]:
treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
p2p = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

Here we implement Functional Maps as an optimization problem

In [ ]:
# Computing descriptors
n_land =  20
n_wks  =  2

n_evals = 20

# Landmarks
step = np.int(np.ceil(v_src.shape[0] / n_land))
a = np.arange(0,v_src.shape[0],step)
landmarks = np.zeros((v_src.shape[0], a.size))
landmarks[a,np.arange(a.size)] = 1

# WKS
d_src = WKS(v_src, f_src, evals_src, evecs_src, n_wks, 7)
d_tar = WKS(v_tar, f_tar, evals_tar, evecs_tar, n_wks, 7)

# Optimization Process
desc_src = np.hstack((landmarks,d_src))
desc_tar = np.hstack((landmarks,d_tar))

# Descriptor normalization
no = np.sqrt(np.diag(np.matmul(M_src.T.__matmul__(desc_src).T, desc_src)))
no_s = np.tile(no.T,(v_src.shape[0],1))
no_t = np.tile(no.T,(v_tar.shape[0],1))
fct_src = np.divide(desc_src,no_s)
fct_tar = np.divide(desc_tar,no_t)

# Coefficents of the obtained descriptors
Fct_src = np.matmul(M_src.T.__matmul__(evecs_src[:, 0:n_evals]).T, fct_src)
Fct_tar = np.matmul(M_tar.T.__matmul__(evecs_tar[:, 0:n_evals]).T, fct_tar)

# The relation between the two constant functions can be computed in a closed form
constFct = np.zeros((n_evals,1))
constFct[0, 0] = np.sign(evecs_src[0, 0] * evecs_tar[0, 0]) * np.sqrt(np.sum(M_tar)/np.sum(M_src))

# Different way to compute Laplacian commutativity
# Dlb = (np.tile(evals_src[0:n_evals], (n_evals, 1)) - np.tile(evals_tar[0:n_evals].T, (n_evals, 1)))**2
# Dlb = np.float32(Dlb/tf.reduce_sum((Dlb**2)))

Play with different energy weights.

In [ ]:
# Energy weights
a = 1e-1 # Descriptors preservation
c = 1e-8 # Commutativity with Laplacian


# Define tensorflow objects
fs = tf.constant(Fct_src, dtype=tf.float32)
ft = tf.constant(Fct_tar, dtype=tf.float32)
evals = tf.constant(tf.linalg.tensor_diag(np.reshape(np.float32(evals_src[0:n_evals]), (n_evals,))), dtype=tf.float32)
evalt = tf.constant(tf.linalg.tensor_diag(np.reshape(np.float32(evals_tar[0:n_evals]), (n_evals,))), dtype=tf.float32)

# Initialize C
C_ini = np.zeros((n_evals,n_evals))
C_ini[0,0]=constFct[0,0]
C = tf.Variable(tf.zeros((n_evals,n_evals), dtype=tf.float32))
C.assign(C_ini)

# Optimizer
adam = tf.keras.optimizers.Adam(1e-1) # Optimization technique
trainable_vars = [C]

# Optimization
for i in np.arange(0,1000):
    with tf.GradientTape() as tape:
        loss1 = a * tf.reduce_sum(((tf.matmul(C, fs) - ft) ** 2)) / 2 # Descriptor preservation
        loss2 = c * tf.reduce_sum((tf.matmul(C, evals) - tf.matmul(evalt,C))**2) #tf.reduce_sum(((C ** 2) * Dlb) / 2)  # Commute with Laplacian
        #loss3 = 1e-6 *  tf.reduce_sum(tf.square(tf.matmul(tf.transpose(C),C) - tf.eye(n_evals))) # Orthonormal C
        loss = loss1  + loss2# + loss3

    # Apply gradient
    grad = tape.gradient(loss,trainable_vars)
    tmp = adam.apply_gradients(zip(grad,trainable_vars))

C = C.numpy()
plt.imshow(C)
plt.colorbar()
plt.show()

treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
p2p = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_src - v_src[p2p,:]))
print(err)

A common post-processing is ICP refinement. The main idea is that we can think to our spectral embedding as N-dimensional pointclouds. If my change of basis $C$ brings the two close enough, we can try to improve the alignment using the rigid registration algorithm called Iterative Closest Point (ICP). 
It is based on two steps. Given a pointcloud $M$ that I want to align to $N$:
- Computing the Nearest Neighbor matching between $M$ and $N$
- Computing the optimal roto-translation via SVD that align $M$ with $N$ using the given NN correspondence
- Iterate until convergence

It is prone to get stuck in local-minima, but if the pointclouds are alrady close enough, it can significatly improve the solution.

In [ ]:
print('ICP refine...')
for k in np.arange(0,5):
    treesearch = scipy.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals],C))
    matches = treesearch.query(evecs_src[:,0:n_evals], k=1)[1]
    W = np.linalg.lstsq(evecs_src[matches, 0:n_evals],evecs_tar[:, 0:n_evals])[0]
    d = np.linalg.svd(W)
    C_ICP = np.matmul(np.matmul(d[0], np.eye(n_evals)), d[2])

plt.imshow(C_ICP)
plt.colorbar()
plt.show()

treesearch = sp.spatial.cKDTree(evecs_tar[:,0:n_evals])
p2p = treesearch.query(np.matmul(evecs_src[:,0:n_evals], C_ICP), k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)

Finally, here we anticipate the ZoomOut refinement that you will see tomorrow. This let to compute a high dimensional $C$. It is based on two iterative steps.Given a $C$ of dimension $n \times n$:
- Convert it into a dense correspondence
- Convert the dense correspondence into a $C$ of dimension $(n+1) \times (n+1)$ 

These two simple steps produce an impressive improvement in the matching quality.

Note: it can be applied to refine _any_ correspondence, also not directly obtained from Functional Maps.

In [ ]:
# More iterations => bigger map => better quality
n_iter = 5

# ZOOMOUT
C_iter = C_ICP
for i in np.arange(0,n_iter):
  # 1) Convert into dense correspondence
  treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals+i], C_iter.T))
  p2p = treesearch.query(evecs_src[:,0:n_evals+i], k=1)[1]

  #2) Convert into C of dimension (n+1) x (n+1)
  C_iter = np.matmul(np.linalg.pinv(evecs_src[:,0:n_evals+i+1]),evecs_tar[p2p,0:n_evals+i+1])

In [ ]:
plt.imshow(C_iter)
plt.colorbar()
plt.show()
treesearch = sp.spatial.cKDTree(np.matmul(evecs_tar[:,0:n_evals+n_iter],C_iter.T))
p2p = treesearch.query(evecs_src[:,0:n_evals+n_iter], k=1)[1]
funz_src = funz_tar[p2p]
plot_pair(v_src, v_tar, f_src, f_tar, funz_src,funz_tar)
err = np.sum(np.square(v_tar - v_tar[p2p,:]))
print(err)